# Step 6 - Paralogous Pairwise Tests
-- Alex Warwick Vesztrocy, February 2024

This notebook contains the code to run the inparalogue pairwise Pearson's correlation ($\rho$) and tissue specificity ($\tau$) tests.

In [16]:
from tqdm.auto import tqdm
from scipy.stats import spearmanr, pearsonr
import os
import numpy as np
import pandas as pd

from parallel_pandas import ParallelPandas
ParallelPandas.initialize()

In [12]:
# species that have enough data and are in PANTHER
dataset = {'animals': {'BOVIN','CHICK','DROME','FELCA','GORGO','HORSE','HUMAN','LEPOC','MACMU','MONDO','MOUSE','ORNAN','ORYLA','PANTR','PIG','RAT'},
           'plants': {'AMBTC','ARATH','BRANA','CAPAN','CUCSA','HELAN','MAIZE','MANES','MARPO','MEDTR','ORYSJ','PHYPA','POPTR','SELML','SETIT','SOLLC','SOLTU','SOYBN','VITVI','WHEAT'}}

In [13]:
pairs_df = pd.read_csv('./results/pairwise_tests.tsv.gz', sep='\t')

In [14]:
pairs = {sp: zdf for (sp, zdf) in pairs_df[pairs_df.species.isin(dataset['animals'] | dataset['plants'])].groupby('species')}

---

### How many tests can we perform?

In [5]:
MAP_FN = './results/expr/gene_map.h5'

In [6]:
res = []
for sp in (sorted(dataset['animals']) + sorted(dataset['plants'])):
    sp_map = pd.read_hdf(MAP_FN, sp)
    have_data = set(sp_map.UniProtKB)

    zdf = pairs[sp]
    
    can_test = sum(zdf.ldo_gene.isin(have_data) & zdf.mdo_gene.isin(have_data))

    res.append((sp, can_test, len(zdf), '{:.02%}'.format(can_test/len(zdf))))

In [7]:
test_stats_df = pd.DataFrame(res, columns=['species', 'no. possible', 'total tests', '% possible'])
test_stats_df.to_csv('./results/tables/test_stats_possible.tsv', sep='\t', index=False)
test_stats_df

,species,no. possible,total tests,% possible
0,BOVIN,7035,9855,71.39%
1,CHICK,4023,5796,69.41%
2,DROME,4285,4501,95.20%
3,FELCA,5359,6829,78.47%
4,GORGO,5306,7594,69.87%
5,HORSE,6365,8042,79.15%
6,HUMAN,6677,7239,92.24%
7,LEPOC,5724,6356,90.06%
8,MACMU,6494,7243,89.66%
9,MONDO,6678,8211,81.33%


---
## Paralogous Expression Similarity

Compute pearson correlation coefficients for the expression values for each of the pairs, for each of the average expression methods.

In [8]:
EXPR = './results/expr'
TAU = './results/tau'
    

def load_expr(sp, expr_fn, tau_fn, map_fn, genes=None):
    # load the expression data, into a consistent format.
    map_df = pd.read_hdf(map_fn, sp)

    if genes is not None:
        map_df = map_df[map_df['UniProtKB'].isin(genes)]

    # load expression data
    expr_df = pd.read_hdf(expr_fn, sp)

    # temporarily flatten
    mi = expr_df.columns.copy()
    expr_df.columns = list(range(len(expr_df.columns)))
    
    # merge and apply transformation
    expr_df = pd.merge(expr_df, map_df, left_on='gene_id', right_on='xref').set_index('UniProtKB')[expr_df.columns]
    expr_df = expr_df.apply(np.arcsinh)

    # regroup the tissue names
    expr_df.columns = mi
    
    # load the tau values
    tau_df = pd.read_hdf(tau_fn, sp)
    tau = pd.merge(pd.DataFrame({'tau': tau_df}), map_df, left_on='gene_id', right_on='xref').set_index('UniProtKB')['tau'].to_dict()

    return (expr_df, tau)


def compute_expr_sim(pair_df, expr_df, sim_func):
    return pair_df.p_apply(lambda x: sim_func(expr_df, x.ldo_gene, x.mdo_gene), axis=1)

def compute_pearson_(expr_df, x, y):
    if x in expr_df.index and y in expr_df.index:
        e_x = expr_df.loc[x].values
        e_y = expr_df.loc[y].values
        if np.all(e_x == e_x[0]) or np.all(e_y == e_y[0]):
            return (None, None)
        return pearsonr(e_x, e_y)
    else:
        return (None, None)

---
## Run for each species

In [9]:
databases = {'animals': [('animal_categorised_tpm_expr.h5', 'animal_categorised_tau.h5', 'animal_categorised'),
                         ('animal_maincat_tpm_expr.h5', 'animal_maincat_tau.h5', 'animal_maincat'),
                         ('animal_tpm_expr.h5', 'animal_alldata_tau.h5', 'animal_alldata')],
             'plants': [('plant_general_tpm_expr.h5', 'plant_general_tau.h5', 'plant_general'),
                        ('plant_specific_tpm_expr.h5', 'plant_specific_tau.h5', 'plant_specific')]}

In [17]:
RES_PATH = './results/correlation'
!mkdir -p {RES_PATH}

for (sp, zdf) in tqdm(pairs.items()):
    if sp in dataset['animals']:
        db_key = 'animals'
    elif sp in dataset['plants']:
        db_key = 'plants'
    else:
        raise ValueError('unknown species {}'.format(sp))

    genes_to_load = (set(zdf.ldo_gene) | set(zdf.mdo_gene))

    # run for each database
    for (expr_fn, tau_fn, key) in databases[db_key]:
        print(sp, key
)
        (expr_df, tau_expr) = load_expr(sp,
                                        expr_fn=f'{EXPR}/{expr_fn}',
                                        tau_fn=f'{TAU}/{tau_fn}',
                                        map_fn=f'{EXPR}/gene_map.h5',
                                        genes=genes_to_load)
        pcc = compute_expr_sim(zdf, expr_df, compute_pearson_)

        zdf['pcc_stat'] = pcc.apply(lambda x: x[0])
        zdf['pcc_p'] = pcc.apply(lambda x: x[1])

        zdf['ldo_tau'] = zdf['ldo_gene'].apply(lambda x: tau_expr.get(x, None))
        zdf['mdo_tau'] = zdf['mdo_gene'].apply(lambda x: tau_expr.get(x, None))

        zdf.to_hdf(f'{RES_PATH}/{key}.h5', sp)

  0%|          | 0/36 [00:00<?, ?it/s]

AMBTC plant_general


<LAMBDA> DONE:   0%|          | 0/6881 [00:00<?, ?it/s]

AMBTC plant_specific


<LAMBDA> DONE:   0%|          | 0/6881 [00:00<?, ?it/s]

...

<LAMBDA> DONE:   0%|          | 0/11769 [00:00<?, ?it/s]

WHEAT plant_general


<LAMBDA> DONE:   0%|          | 0/64988 [00:00<?, ?it/s]

WHEAT plant_specific


<LAMBDA> DONE:   0%|          | 0/64988 [00:00<?, ?it/s]